# Exercise 2 - Selenium test

The aim of this exercise is to test if all the necessary libraries and drivers are installed correctly. **Before classes** make sure that everything works correctly and in case of any problems and ask your mentor for help.

Download one of the drivers (and the corresponding browser if you do not have it yet):
  - [This is where the driver for Firefox is](https://github.com/mozilla/geckodriver/releases)
  - [This is where the driver for Chrome is](https://chromedriver.chromium.org/downloads)

Extract the `geckodriver`, `geckodriver.exe`, `chromedriver` or `chromedriver.exe` file and put it in the folder **right next to this exercise**.

If everything works as it should, eventually you will see the rates for five currencies read from the website of the National Bank of Poland.

> During the course you're going to need Selenium version 4.*+ - if you use an older version your notebook may not open correctly. You'll find an extra comment in the first cell below.

## CSS Selectors

When you use Selenium, you are going to need CSS selectors: you can learn them quickly while having fun **[playing the flukeout.github.io game](https://flukeout.github.io/)**.

In [3]:
!pip show selenium  # check installed Selenium version

Name: selenium
Version: 4.36.0
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache-2.0
Location: C:\Users\mjemelka\AppData\Local\anaconda3\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 
---
Name: selenium
Version: 4.36.0
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache-2.0
Location: C:\Users\mjemelka\AppData\Local\anaconda3\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 


your Selenium version should be 4.*+; If it isn't uncomment the line below and restart the notebook:

In [4]:
# uncomment as needed (see above)
!pip install --upgrade --force-reinstall selenium

  Using cached selenium-4.36.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.31.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-2.0.0-cp313-cp313-win_amd64.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metad

In [5]:
import selenium
print("Selenium version:", selenium.__version__)


Selenium version: 4.36.0


## Importing required libraries

In [1]:
from selenium.webdriver import Firefox, Chrome
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.common.exceptions import InvalidArgumentException

from selenium.webdriver.common.by import By

import os

In [2]:
def get_browser():
  browser = None
  service = None
  if os.path.exists('./geckodriver.exe'):
      service = FirefoxService('./geckodriver.exe')
      browser = Firefox(service=service)
      
  elif os.path.exists('./geckodriver'):
      service = FirefoxService('./geckodriver')
      browser = Firefox(service=service)
      
  elif os.path.exists('./chromedriver.exe'):
      service = ChromeService('./chromedriver.exe')
      browser = Chrome(service=service)
      
  elif os.path.exists('./chromedriver'):
      service = ChromeService('./chromedriver')
      browser = Chrome(service=service)
  
  return browser


In [3]:
try:
    browser = get_browser()
except InvalidArgumentException:
    print("Read the comment in the next cell")

if browser is None:
    print('None of the listed files found!')
    print(f'Make sure that the file is in the working directory of the notebook: "{os.getcwd()}"')

None of the listed files found!
Make sure that the file is in the working directory of the notebook: "C:\Users\mjemelka"


In [4]:
# --- Selenium test: načtení 5 kurzů NBP (Bankier) ---
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

URL = "https://www.bankier.pl/waluty/kursy-walut/nbp"

opts = Options()
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-gpu")
opts.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=opts)

def get_cell_text(row_el, idx, default=""):
    """Bezpečně vrátí text z td[idx] nebo default."""
    try:
        return row_el.find_elements(By.CSS_SELECTOR, "td")[idx].text.strip()
    except Exception:
        return default

try:
    driver.get(URL)

    # počkej na načtení řádků tabulky
    rows = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr"))
    )

    # projdi prvních 5 řádků a vytiskni údaje
    counter = 0
    for row in rows[:5]:
        # typická struktura: [Waluta, Kod, Ilość, Kurs średni, Zmiana, Data]
        name = get_cell_text(row, 0)
        code = get_cell_text(row, 1)
        rate = get_cell_text(row, 3)  # "Kurs średni"
        if name and code and rate:
            print(f"{name} ({code}) is now worth {rate} PLN")
            counter += 1

    # vyhodnocení testu (stejná logika jako v zadání)
    if counter == 5:
        print("Five currency rates checked - everything works!")
    else:
        print("ERROR!!! We expected to get five exchange rates - something went wrong!")
        print(f"Found {counter}/5 rows. The page structure may have changed.")

finally:
    driver.quit()


Strefa Euro (4,2559) is now worth 1 EUR PLN
Stany Zjednoczone (3,6253) is now worth 1 USD PLN
Wielka Brytania (4,8784) is now worth 1 GBP PLN
Szwajcaria (4,5527) is now worth 1 CHF PLN
Czechy (0,1754) is now worth 1 CZK PLN
Five currency rates checked - everything works!


### NOTE

If the previous cell has an `InvalidArgumentException` open Firefox using the additional configuration: 
```python
from selenium.webdriver.firefox.options import Options
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe' # This is the path where your Firefox should be installed; edit it if necessary

options = Options() 
options.binary = binary

service = FirefoxService('./geckodriver.exe')
browser = Firefox(service=service, options=options)
```
--------
the code above should replace this one:
```python
service = FirefoxService('./geckodriver.exe')
browser = Firefox(service=service)
```

If you still have problems, contact your mentor.
> If you used this code to configure your browser you need to remember about it during classes!

In [5]:
if browser is not None:
    browser.get('https://www.nbp.pl/')
    table = browser.find_element(by=By.CSS_SELECTOR, value='.row.exchange__table')
    currencies = table.find_elements(by=By.CSS_SELECTOR, value='div.exchange__table-col-1')
    rates = table.find_elements(by=By.CSS_SELECTOR, value='div.exchange__table-col-2')

    counter = 0
    if len(currencies) == len(rates):
        for i, item in enumerate(currencies):
            currency = currencies[i].text
            rate = rates[i].text
            print(f'{currency} is now worth {rate} PLN')
            counter += 1

    browser.quit()
    
    if counter == 5:
        print('Five currency rates checked - everything works!')
    else:
        print('ERROR!!! We expected to get five exchange rates - something went wrong!')
else:
    print("browser variable is not defined correctly. Check if running the script launches a new browser window.")

browser variable is not defined correctly. Check if running the script launches a new browser window.


In [6]:
# --- Selenium test: načtení 5 kurzů NBP (Bankier) ---
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

URL = "https://www.bankier.pl/waluty/kursy-walut/nbp"

opts = Options()
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-gpu")
opts.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=opts)

def get_cell_text(row_el, idx, default=""):
    """Bezpečně vrátí text z td[idx] nebo default."""
    try:
        return row_el.find_elements(By.CSS_SELECTOR, "td")[idx].text.strip()
    except Exception:
        return default

try:
    driver.get(URL)

    # počkej na načtení řádků tabulky
    rows = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr"))
    )

    # projdi prvních 5 řádků a vytiskni údaje
    counter = 0
    for row in rows[:5]:
        # typická struktura: [Waluta, Kod, Ilość, Kurs średni, Zmiana, Data]
        name = get_cell_text(row, 0)
        code = get_cell_text(row, 1)
        rate = get_cell_text(row, 3)  # "Kurs średni"
        if name and code and rate:
            print(f"{name} ({code}) is now worth {rate} PLN")
            counter += 1

    # vyhodnocení testu (stejná logika jako v zadání)
    if counter == 5:
        print("Five currency rates checked - everything works!")
    else:
        print("ERROR!!! We expected to get five exchange rates - something went wrong!")
        print(f"Found {counter}/5 rows. The page structure may have changed.")

finally:
    driver.quit()


Strefa Euro (4,2559) is now worth 1 EUR PLN
Stany Zjednoczone (3,6253) is now worth 1 USD PLN
Wielka Brytania (4,8784) is now worth 1 GBP PLN
Szwajcaria (4,5527) is now worth 1 CHF PLN
Czechy (0,1754) is now worth 1 CZK PLN
Five currency rates checked - everything works!
